<a href="https://colab.research.google.com/github/ukhyun-lee/stock_market_analysis/blob/main/%EC%A4%91%EA%B8%89/9%EC%9D%BC%EC%B0%A8_%EC%8B%A4%EC%8A%B53.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.compose import ColumnTransformer

In [7]:
from sklearn.datasets import fetch_openml

In [12]:
X, y = fetch_openml('titanic', version=1, as_frame=True, return_X_y=True, parser='pandas')

In [13]:
X

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   pclass     1309 non-null   int64   
 1   name       1309 non-null   object  
 2   sex        1309 non-null   category
 3   age        1046 non-null   float64 
 4   sibsp      1309 non-null   int64   
 5   parch      1309 non-null   int64   
 6   ticket     1309 non-null   object  
 7   fare       1308 non-null   float64 
 8   cabin      295 non-null    object  
 9   embarked   1307 non-null   category
 10  boat       486 non-null    object  
 11  body       121 non-null    float64 
 12  home.dest  745 non-null    object  
dtypes: category(2), float64(3), int64(3), object(5)
memory usage: 115.4+ KB


In [22]:
X.sex

0       female
1         male
2       female
3         male
4       female
         ...  
1304    female
1305    female
1306      male
1307      male
1308      male
Name: sex, Length: 1309, dtype: category
Categories (2, object): ['female', 'male']

In [48]:
X['embarked'].value_counts(dropna=False)

S      914
C      270
Q      123
NaN      2
Name: embarked, dtype: int64

In [49]:
X[X.embarked.isna()]

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
168,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,6,NaN,NaN
284,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,6,NaN,"Cincinatti, OH"


In [44]:
from sklearn.preprocessing import OneHotEncoder

In [45]:
ohe = OneHotEncoder(categories=[['S','C','Q']], handle_unknown='ignore')

In [47]:
ohe.fit_transform(X[['embarked']]).toarray()[168]

array([0., 0., 0.])

In [15]:
from sklearn.pipeline import Pipeline

In [18]:
from sklearn.impute import SimpleImputer

In [20]:
from sklearn.preprocessing import StandardScaler

In [24]:
num_pipe = Pipeline([('imputer',SimpleImputer()),('std',StandardScaler())])

In [51]:
cat_pipe = Pipeline([('encoder', OneHotEncoder()),('select', SelectPercentile(chi2, percentile=50))])

In [53]:
preprocess = ColumnTransformer([('num', num_pipe, ['age','fare']),
                                ('cat', cat_pipe, ['embarked','sex','pclass'])])

In [55]:
from sklearn.linear_model   import LogisticRegression

In [56]:
pipe = Pipeline([('pre',preprocess), ('lr', LogisticRegression()) ])

In [63]:
pipe.get_params()

{'memory': None,
 'steps': [('pre', ColumnTransformer(transformers=[('num',
                                    Pipeline(steps=[('imputer', SimpleImputer()),
                                                    ('std', StandardScaler())]),
                                    ['age', 'fare']),
                                   ('cat',
                                    Pipeline(steps=[('encoder', OneHotEncoder()),
                                                    ('select',
                                                     SelectPercentile(percentile=50,
                                                                      score_func=<function chi2 at 0x7faff0a92b00>))]),
                                    ['embarked', 'sex', 'pclass'])])),
  ('lr', LogisticRegression())],
 'verbose': False,
 'pre': ColumnTransformer(transformers=[('num',
                                  Pipeline(steps=[('imputer', SimpleImputer()),
                                                  ('std', Stand

In [64]:
from sklearn.model_selection import GridSearchCV

In [65]:
grid = GridSearchCV(pipe, {'pre__num_imputer_strategy':['mean','median'],
                           'pre__cat__select':[10,20,30,40,50]})

In [57]:
from sklearn.compose import make_column_selector

In [58]:
make_column_selector


sklearn.compose._column_transformer.make_column_selector

In [66]:
preprocess = ColumnTransformer([('num', num_pipe, make_column_selector(dtype_exclude='category')),
                                ('cat', cat_pipe, make_column_selector(dtype_include='category'))])

In [67]:
make_column_selector(dtype_include='category')(X)

['sex', 'embarked']

##select_dtype

In [59]:
import seaborn as sns
tips = sns.load_dataset('tips')

In [60]:
tips.select_dtypes('category')

,sex,smoker,day,time
0,Female,No,Sun,Dinner
1,Male,No,Sun,Dinner
2,Male,No,Sun,Dinner
3,Male,No,Sun,Dinner
4,Female,No,Sun,Dinner
...,...,...,...,...
239,Male,No,Sat,Dinner
240,Female,Yes,Sat,Dinner
241,Male,Yes,Sat,Dinner
242,Male,No,Sat,Dinner


In [61]:
tips.describe(include=['category'])

,sex,smoker,day,time
count,244,244,244,244
unique,2,2,4,2
top,Male,No,Sat,Dinner
freq,157,151,87,176
